In [34]:
setwd('/nas/cee-water/cjgleason/colin/neobam/')
index=32
source('run_neobam.R')
    

[1] "Read was successful..."


recompiling to avoid crashing R session

code for methods in class “Rcpp_stan_fit4model2fb855a73ed97_neobam_stan_engine” was not checked for suspicious field assignments (recommended package ‘codetools’ not available?)

code for methods in class “Rcpp_stan_fit4model2fb855a73ed97_neobam_stan_engine” was not checked for suspicious field assignments (recommended package ‘codetools’ not available?)

Warning message:
“There were 1216 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“There were 66 transitions after warmup that exceeded the maximum treedepth. Increase max_treedepth above 10. See
https://mc-stan.org/misc/warnings.html#maximum-treedepth-exceeded”
Warning message:
“There were 3 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pa

$posterior_Q
[1]  65.01211  59.65653  44.79909  64.06305  57.66884 161.63445

$posterior_Q_sd
[1] 1.055052

$posteriors
$posteriors$r
$posteriors$r$mean
 [1] 0.3050296 0.3165319 0.3013994 0.3005926 0.3015133 0.3023896 0.3031663
 [8] 0.3024178 0.3017783 0.3024777 0.3044541 0.3019141 0.3004608 0.3009039
[15] 0.3005364 0.3012390 0.3005401 0.3015571 0.3222794 0.3149300 0.3529670
[22] 0.3484220 0.3079747 0.3077839 0.4499323 0.3417087 0.3043743 0.3019383
[29] 0.5015771

$posteriors$r$sd
[1] 5.156832e-05


$posteriors$logn
$posteriors$logn$mean
 [1] -3.440133 -3.777569 -3.184030 -3.008216 -4.578500 -3.478340 -3.608731
 [8] -3.281750 -3.137184 -4.217634 -3.422637 -3.417660 -3.002793 -3.140406
[15] -4.601670 -3.328507 -3.009388 -3.036120 -3.507483 -3.569632 -3.596802
[22] -3.510405 -3.445222 -3.381788 -4.029069 -3.661813 -3.321044 -3.058895
[29] -3.521244

$posteriors$logn$sd
[1] 0.00435349


$posteriors$logWb
$posteriors$logWb$mean
 [1] 6.034107 6.468579 5.899754 6.354809 6.423541 6.429950 6.5